# Versions

* Version 1: CV:-0.0.4875 LB:-

# Imports

In [1]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#General
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from glob import glob
import os

#Sklearn
from sklearn.model_selection import train_test_split , KFold

#Pytorch
import torch 
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

#Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import seed_everything , Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

#Hugging Face
from transformers import AutoModel , AutoConfig , AutoTokenizer ,  AdamW, get_linear_schedule_with_warmup,get_constant_schedule_with_warmup,get_cosine_schedule_with_warmup

In [3]:
paths = glob("../input/commonlitreadabilityprize/*.csv")
paths = sorted(paths)

df_ss = pd.read_csv(paths[0])
df_test = pd.read_csv(paths[1])
df_train = pd.read_csv(paths[2])

# Configs

In [4]:
class config:
    seed = 123
    batch_size = 16
    epochs = 20
    transformer_name = "roberta-base"
    transformer_path = "../input/huggingface-roberta/roberta-base"
    max_len = 250
    learning_rate = 2e-5
    save_dir = "./result"

seed_everything(config.seed)

123

In [5]:
if not os.path.exists(config.save_dir):
    os.makedirs(config.save_dir)

# Dataset

In [6]:
class CLRDataset:
    def __init__(self, name , dataset):
        self.tokenizer = AutoTokenizer.from_pretrained(name)
        self.max_len = config.max_len
        
        self.excerpt = dataset['excerpt'].to_numpy()
        self.target = dataset['target'].to_numpy()
        

    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self , idx):
        text = self.excerpt[idx]
        target = self.target[idx]
        tokenized_text = self.tokenizer(text, truncation = True , padding = "max_length" , max_length= self.max_len )
        
        return {'input_ids': torch.tensor(tokenized_text['input_ids'], dtype = torch.long),
                'attention_mask' : torch.tensor(tokenized_text['attention_mask'] , dtype = torch.long),
               'target': torch.tensor([target],dtype = torch.float)}

# Model

In [7]:
def loss_function(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))

In [8]:
class AttentionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(768,512)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(512,1)
        self.softmax = nn.Softmax(dim = 1)
    
    def forward(self,input_tensors):
        x = self.linear(input_tensors)
        x = self.tanh(x)
        x = self.linear2(x)
        x = self.softmax(x)
        
        return x

class TransformerModel(nn.Module):
    def __init__(self,name):
        super().__init__()
        self.transformer_config = AutoConfig.from_pretrained(name)
        self.transformer_config.update({"output_hidden_states":True, "hidden_dropout_prob": 0.0,"layer_norm_eps": 1e-7}) 
        self.transformer_model = AutoModel.from_pretrained(name , config = self.transformer_config)
        
    def forward(self,input_ids, attention_mask):
        transformer_output = self.transformer_model(input_ids = input_ids , attention_mask = attention_mask)
        transformer_output = transformer_output.hidden_states[-1]
        
        return transformer_output

class RegressionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(768,1)
    
    def forward(self,input_tensors):
        x = self.linear(input_tensors)
        
        return x

class CLRModel(pl.LightningModule):
    def __init__(self,name , train , validation):
        super().__init__()
        self._train_dataloader = train
        self._val_dataloader = validation
        self._test_dataloader = validation
        self.name = name
        self.transformer_model = TransformerModel(self.name)
        self.regression_head = RegressionHead()
        self.attention_head = AttentionHead()
        self.save_hyperparameters()
    
    def forward(self,input_ids , attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids , attention_mask)
        weights = self.attention_head(last_layer_hidden_states) 
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)  
        output = self.regression_head(context_vector)
        
        return output
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target = batch['target']
        

        output = self.forward(input_ids , attention_mask)
        loss = loss_function(output,target) 
        self.log('train_loss', loss , prog_bar=True)

        return {'loss': loss}

    def train_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print(f'epoch {trainer.current_epoch} training loss {avg_loss}')
        return {'train_loss': avg_loss}    
    
    
    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target = batch['target']
        

        output = self.forward(input_ids , attention_mask)
        loss = loss_function(output,target) 
        self.log('val_loss', loss, prog_bar=True)
        
        return {'val_loss': loss}
    

    def validation_epoch_end(self, outputs):

        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print(f'epoch {trainer.current_epoch} validation loss {avg_loss}')
        return {'val_loss': avg_loss}
    
    
    def test_step(self, batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target = batch['target']

        output = self.forward(input_ids , attention_mask)
        loss = loss_function(output,target) 
        self.log('test_loss', loss)
        return {'test_loss': loss}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        print(f'epoch {trainer.current_epoch} test loss {avg_loss}')
        return {'test_loss': avg_loss}
    
    def train_dataloader(self):
        return self._train_dataloader
    
    def val_dataloader(self):
        return self._val_dataloader
    
    def test_dataloader(self):
        return self._val_dataloader
    
    def configure_optimizers(self):
        """optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
        lr_scheduler = get_constant_schedule_with_warmup(optimizer, 100)"""
        named_parameters = list(model.named_parameters())    
    
        roberta_parameters = named_parameters[:197]    
        attention_parameters = named_parameters[199:203]
        regressor_parameters = named_parameters[203:]

        attention_group = [params for (name, params) in attention_parameters]
        regressor_group = [params for (name, params) in regressor_parameters]
        parameters = []
        parameters.append({"params": attention_group})
        parameters.append({"params": regressor_group})

        for layer_num, (name, params) in enumerate(roberta_parameters):
            weight_decay = 0.0 if "bias" in name else 0.01

            lr = 2e-5

            if layer_num >= 69:        
                lr = 5e-5

            if layer_num >= 133:
                lr = 1e-4

            parameters.append({"params": params,
                               "weight_decay": weight_decay,
                               "lr": lr})
        
        optimizer = AdamW(parameters)            
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_training_steps= config.epochs * len(train_dataloader),
            num_warmup_steps=50)
            

        
        return [optimizer], [scheduler]
    
    


In [9]:
from pytorch_lightning.callbacks import ProgressBarBase ,ProgressBar


class EpochProgressBar(ProgressBarBase):

    def __init__(self):
        super().__init__()
        self.bar = None

    def on_train_start(self, trainer, pl_module):
        self.bar = tqdm(
            desc='Epoch',
            leave=False,
            dynamic_ncols=True,
            total=trainer.max_epochs,
        )

    def on_train_epoch_end(self, *args, **kwargs):
        self.bar.update(1)

class LitProgressBar(ProgressBar):
    def __init__(self):
        super().__init__()
        
    def on_train_epoch_start(self, trainer, pl_module):
        if trainer.current_epoch:
            print()
        super().on_train_epoch_start(trainer, pl_module)

In [11]:
scores=[]
iterations = 1
kfold = KFold(n_splits=5, shuffle= True , random_state = config.seed)
for train_idx, test_idx in kfold.split(df_train):
    print("************** iteration",iterations,"**************")
    
    train_data = df_train.loc[train_idx]
    validation_data = df_train.loc[test_idx]
    
    train = CLRDataset(config.transformer_path,train_data)
    valid = CLRDataset(config.transformer_path,validation_data)
    
    train_dataloader = DataLoader(train , batch_size = config.batch_size , shuffle = True , num_workers=4,pin_memory=False)
    validation_dataloader = DataLoader(valid , batch_size = config.batch_size , shuffle = False , num_workers=4,pin_memory=False)
    checkpoint_callback = ModelCheckpoint(monitor='val_loss',
                                          dirpath= config.save_dir,
                                      save_top_k=1,
                                      save_last= False,
                                      save_weights_only=True,
                                      filename= f"./fold_{iterations}",
                                      verbose= True,
                                      mode='min')
    
    model = CLRModel(config.transformer_path, train_dataloader , validation_dataloader)
    trainer = Trainer(max_epochs= config.epochs,gpus =1 , progress_bar_refresh_rate=10,callbacks=[checkpoint_callback])
    trainer.fit(model , train_dataloader , validation_dataloader)
    
    #predictions
    print('predicting')
    model_test = CLRModel.load_from_checkpoint(f'result/fold_{iterations}.ckpt')
    a = trainer.test(model_test)
    
    iterations +=1 

************** iteration 1 **************


Some weights of the model checkpoint at ../input/huggingface-roberta/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Validation sanity check: 0it [00:00, ?it/s]

epoch 0 validation loss 1.5043253898620605


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

epoch 19 validation loss 0.46861532330513
predicting


Some weights of the model checkpoint at ../input/huggingface-roberta/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing: 0it [00:00, ?it/s]

epoch 19 test loss 0.46861532330513
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.47284188866615295}
--------------------------------------------------------------------------------


In [13]:
(0.47284188866615295 + 0.4912501275539398 + 0.4916948080062866 + 0.47790032625198364 + 0.5041112899780273)/5

0.4875596880912781